# Продвинутый Python, лекция 6

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## HTTP и иже с ним

Теперь приступим к новым вещам, а именно - к вебом и взаимодействием с ним!

![](https://www.thebozho.com/wp-content/uploads/2017/04/Free-Open-Internet.jpg)

Давайте начнем просто с того, что же такое HTTP? В переводе это HyperText Transfer Protocol, или же протокол для передачи т.н. HyperText (гиепртекст - это текст с наличием ссылок для перехода на другие странички, изи)

Что в таком случае https, который мы сейчас везде видим? Это тот же HTTP, но еще и S (secure) - данные передаются не просто открыто, а через протокол шифрования [TLS](https://habr.com/ru/post/258285/) (не будет вдаваться в подробности, как оно работает, речь не про это)

По сути своей, это модель "клиент-сервер". Клиент дает запрос, сервер отвечает, еще запрос, еще ответ etc хоть до бесконечности. Выглядит максимально просто и понятно

К примеру, когда вы заходите на сайт, вы отправляете запрос по типу "покажи сайт". Сервер такой: "Держи". Потом тыкаете на ссылку, опять запрос etc



Что происходит при открытии сайта?

Например, вы заходите в [питон](https://python.org/). Что необходимо сделать для этого браузеру?

Браузер отправляет запрос на сервер, который расположен по адресу python.org. Удивительно, но это можно сделать даже без браузера (кто бы мог подумать) 

In [ ]:
!brew install telnet
!telnet python.org 80 # утилита, которая осуществляет запрос как клиентская часть

Подсоединились, теперь нам надо отправлять запросы, чтобы получать какие-то ответы. Как выглядят запросы в HTTP?

На самом деле тут все достаточно просто. Любой запрос в HTTP выглядит следующим образом:

```
Метод URI HTTP/1.1\n
Host: hostname\n
\n

```

* Метод - собственно операция, которую хотим осуществить

* URI - путь до документа/страницы, которая нам нужна

* HTTP/1.1 - указываем версию HTTP (современная - 1.1) 

* hostname - название хоста (так как удалённый сервер не обладает никакой информацией о том, какой именно адрес использовался для соединения, то его надо указывать)

На самом деле можно указывать также дополнительные параметры (например, какой у нас User-Agent (браузер) etc)

Базовый метод - это GET (название говорит само за себя). Ну давайте получим информацию (перейдем на Python, не зря же его изучаем)

In [25]:
import telnetlib #библиотека для подключений, будет deprecated

print("connecting...")
tn = telnetlib.Telnet("www.python.org", '80')
msg = "GET / HTTP/1.1\nHost:www.python.org\n\n".encode('ascii') #необходимо, чтобы отправить без пробразования \n в перевод строки внутри строки
tn.write(msg)
print(tn.read_all().decode("ascii"))
print("response received")
tn.close()

connecting...
HTTP/1.1 301 Moved Permanently
Connection: close
Content-Length: 0
Server: Varnish
Retry-After: 0
Location: https://www.python.org/
Accept-Ranges: bytes
Date: Sun, 09 Oct 2022 20:45:58 GMT
Via: 1.1 varnish
X-Served-By: cache-iad-kjyo7100086-IAD
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1665348358.489964,VS0,VE0
Strict-Transport-Security: max-age=63072000; includeSubDomains


response received


Что мы всегда видим в качестве ответа? В первую очередь строку следующего вида:

```
HTTP/1.1 Код Пояснение
```

* HTTP/1.1 - версия HTTP

* Код - [код выполнения запроса](https://developer.mozilla.org/ru/docs/Web/HTTP/Status)

Вот наши любимые:

1. 200 - все ок

2. 4** - вы лохb (сюда относится и 403, 404 - запрещено/не найдено)

3. 5** - мы лохи

* Пояснение - название кода ответа (типа OK, Not Found etc)


Дальше идет тело ответа: что собственно, мы запросили вообще (но никакого response нет, ибо 301)

Какие есть основные методы для HTTP?

* GET - получи инфу

* POST - запость инфу

* PUT - обнови инфу

* DELETE - удали инфу

Если вы решили, что можно взять и тогда удалить Python, то вот нет. Единственное, что обязан обрабатывать веб-сервер - это GET. Остального может не быть (и тогда сервер вас просто не поймет). А еще могут быть всякие другие методы

Понятное дело, что для этого должна быть какая-нибудь удобная библиотека (а не тупо telnet, в который еще как-то странно надо передавать данные). Для этого есть requests!

## Requests

[Requests](https://requests.readthedocs.io/en/latest/api/) - это отличная библиотека для работы с подключениями к сайту. Давайте разбираться на примерах:

In [27]:
import requests

requests.get('https://api.github.com') #аналогия get, получаем ответ

<Response [200]>

Какие атрибуты есть у get?

In [48]:
r = requests.get('https://api.github.com')
print(r.status_code, r.reason) # статус ответа (200 - все ОК)
print('-' * 30)
for i in r.headers.items(): # информация
    print(i[0], ':', i[1])
print('-' * 30)
print(r.encoding)
print('-' * 30)
print(r.text) # что вывели (тело) (в чистом виде можно с помощью .content, text сразу декодирует)
print('-' * 30)
print(r.url) # где находимся
print('-' * 30)
print(r.json) # и можно сразу в виде json

200 OK
------------------------------
Server : GitHub.com
Date : Sun, 09 Oct 2022 22:01:46 GMT
Content-Type : application/json; charset=utf-8
Cache-Control : public, max-age=60, s-maxage=60
Vary : Accept, Accept-Encoding, Accept, X-Requested-With
ETag : W/"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"
X-GitHub-Media-Type : github.v3; format=json
Access-Control-Expose-Headers : ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset
Access-Control-Allow-Origin : *
Strict-Transport-Security : max-age=31536000; includeSubdomains; preload
X-Frame-Options : deny
X-Content-Type-Options : nosniff
X-XSS-Protection : 0
Referrer-Policy : origin-when-cross-origin, strict-origin-when-cross-origin
Content-Security-Policy : default-src 'none'
Content-Encod

Иногда мы хотим делать get не просто по сайту, а еще передавать какие-нибудь параметры. Давайте попробуем:

In [51]:
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'Deep_Python'}, #делаем поиск с помощью параметра q https://docs.github.com/en/rest/search
)
 
# Анализ некоторых атрибутов местонахождения запросов
json_response = response.json()
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')
print(f'Repository description: {repository["description"]}')

Repository name: deep-learning-with-python-notebooks
Repository description: Jupyter notebooks for the code samples of the book "Deep Learning with Python"


In [59]:
for i in range(len(json_response['items'])):
    if json_response['items'][i]['name'] == 'Deep_Python':
        print(i)

21


In [ ]:
json_response['items'][21]

Какие есть функции?

Есть самый основные для HTTP-протокола:

In [62]:
requests.post('https://httpbin.org/post', data={'key':'value'}) #запостить инфу
requests.put('https://httpbin.org/put', data={'key':'value'}) #полная замена инфы (придется указать все, что даже не меняется)
requests.delete('https://httpbin.org/delete') #удалить инфу
requests.head('https://httpbin.org/get') #получить информацию без тела
requests.patch('https://httpbin.org/patch', data={'key':'value'}) #замена инфы, но без указания того, что не надо заменять

<Response [504]>

Иногда для того, чтобы получить доступ, надо сделать авторизацию (аутентификацию), для этого внутри requests есть 

In [65]:
from requests.auth import HTTPBasicAuth #самый простой способ аутентификации, почти везде принимается
from getpass import getpass #библиотека из 2 функций: запрос пароля и запрос пользователя

requests.get(
     'https://api.github.com/user',
     auth=HTTPBasicAuth('user', getpass())
)

··········


<Response [401]>

Давайте себе представим ситуацию: сервер подвис! Что же делать, наш код будет сидеть и ждать, а такого мы не хотим. Внутри requests можно задать время ожидания, который мы используем

In [67]:
from requests.exceptions import Timeout
 
try:
    response = requests.get('https://api.github.com', timeout=1) #устанавливаем максимальный timeout, если не случилось, то он бросает Timeout
except Timeout:
    print('The request timed out')
else:
    print('The request did not time out')

The request did not time out


И последнее, что мы сегодня разберем, так это Session. Что это такое?

Сессия - это возможность сделать несколько обращений, при этом сохранив общие параметры, которые вам нужны (ну, например, если у вас вдруг везде одинаковый логин-пароль, можно его сохранить и использовать для нескольких вызовов)

In [72]:
with requests.Session() as session: 
    session.auth = ('username', getpass())
    session.headers.update({'one': 'true'})
    print(session.headers)
    response = session.get('https://api.github.com')
 
print(response.headers)
print(response.json())

··········
{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'one': 'true'}
{'Server': 'GitHub.com', 'Date': 'Sun, 09 Oct 2022 23:57:32 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '55', 'X-RateLimit-Reset': '1665362812', 'X-RateLimit-Used': '5', 'X-RateLimit-Resource': 'core', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, 

Наверное, вы слышали какую-нибудь фразу типа "REST API". Что это вообще значит?

Давайте разбираться по словам

## REST

REST (Representational State Transfer) - это, в сущности, некоторое соглашения по поводу того, как проектировать наши запросы и как с ними работать (то есть просто принципы, которым все следуют, чтобы было проще работать)

Что в себя включает REST?

1. Клиент-серверная архитектура (есть две части: клиент, который отправляет запросы и сервер, который принимает запросы и обрабатывает их. Эти части разделены). Пример не клиент-серверной архитектуры - блокчейн (клиент и сервер соединены, все самодостаточны)

2. Stateless (обращаясь к серверу, он нигде не запоминает сам запрос. А если храним, то только на стороне клиента)

3. Кэширование (можно использовать кэш, чтобы не обращаться дважды к другим источникам на основе уже ранее показанного)

4. Единообразие интерфейса (сервер возвращает не только ресурс, но и его связи с другими ресурсами и действия, которые можно с ним совершить)

5. Layered system (ни клиент, ни сервер не должны знать о том, как происходит цепочка вызовов дальше своих прямых соседей, то есть линейная система)

6. Code on demand (передаем сам код выполнения, а не только само действие)

Но что более интересно - каждый понимает REST по-своему, что несколько затрудняет нашу жизнь. Изложенное выше - это оригинальное трактование REST, но разные люди могут по-разному ее определять. Почему? Потому что это соглашения, а не железное правило. Но самое главное, что REST - это всегда про HTTP протоколы (и еще обычно говорят, что для REST всегда должны быть методы GET, DELETE, POST, PUT)

Теперь перейдем ко второму слову - API

## API

API (Application programming interface) - некоторое соглашение (или декларация) о том, какие функции есть, как к ним обращаться и как работать. По сути - документация

В качестве примера: https://requests.readthedocs.io/en/latest/api/ - вот это API

Тыкаем вызов функции - вызывается API, допустим pandas. А pandas еще внутри вызывает API numpy (они очень связаны). А если вы вызываете в seaborn отрисовку графика через данные в pandas, то вы понимаете, что происходит

И в совокупности REST API - это обмен данными между API по REST принципу. Все просто! Зачем так делать? Для простоты взаимодействия (все следуют общим правилам - есть общая система, где не надо вникать - успех!)

Зачем нам это все знать? Ну вот допустим, что вы решили сделать сайт, для него же надо сделать API, чтобы клиент мог обращаться, тыкаться и работать с ним. Вот для этого

На семинаре мы поговорим про Telegram API (и ботика напишем простого), а дальше будет еще лучше)

## Птица дня 

![](https://mtdata.ru/u16/photoE47C/20790968880-0/original.jpeg)

А это кетцаль (тревога: это не попугай!). Священная птица для майя и ацтеков (и в целом для мезоамериканских народов, не зря же Кетцалькоатль)

Их ловили, но никогда не убивали. Единственное, что от них было нужно - это красивые перья, которые использовались в качестве украшений и для ритуалов. Их перья были символом власти и сверхъестественного. Местные жители считают, что в неволе птица умирает от разрыва сердца

Во времена испанского правления в кетцаль был символом свободы для местных жителей и даже есть легенда, что его грудь и брюхо окрашены кровью павших борцов за независимость. И сейчас в Гватемале это государственный символ (и валюта)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Feather_headdress_Moctezuma_II.JPG/1920px-Feather_headdress_Moctezuma_II.JPG)